## Extractng First and last name from the Egyptian NID

### Imports and Instaling

In [2]:
! pip install pytesseract

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!sudo apt-get install tesseract-ocr

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 24 not upgraded.
Need to get 4,850 kB of archives.
After this operation, 16.3 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1 [1,598 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal/universe amd64 tesseract-ocr amd64 4.1.1-2build2 [262 kB]
Fetched 4,850 kB in 1s (3,370 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/Fro

In [28]:
!sudo apt-get install tesseract-ocr-ara  #for arabic

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr-ara is already the newest version (1:4.00~git30-7274cfa-1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


In [4]:
from ArabicOcr import arabicocr
import pytesseract
from pytesseract import Output
from google.colab.patches import cv2_imshow
import cv2
import numpy as np

In [5]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [6]:
import os
from os import listdir
from os.path import isfile, join

img_paths = {} #Images paths 
img_names = [] #Images names

for root, dirs, files in os.walk("/content/gdrive/MyDrive/NTI-Project/Original ID"):
   path = root.split(os.sep)
   for index, file in enumerate(files):
      img_names = [ f for f in listdir(root) if isfile(join(root,f)) ]
      img_paths[index] = join(root,img_names[index])

In [7]:
#img_paths
#img_names

In [8]:
from PIL import Image
from pathlib import Path

In [9]:
firstname_box  = (465,165, 995,233)   #Dimentions of the first name in the origibal ID
lastname_box   = (465,230, 995,310)   #Dimentions of the last name in the origibal ID
nid_box        = (430,475 , 1020,575) #Dimentions of the national ID number name in the origibal ID

for i in range(len(img_paths)):
  img = Image.open(img_paths[i])
  
  #Crop the first name, last name and ID to different pictures
  firstname = img.crop(firstname_box)
  lastname  = img.crop(lastname_box)
  nid       = img.crop(nid_box)

  #Save cropped pictures
  firstname_path = "/content/gdrive/MyDrive/NTI-Project/First Names/" +  img_names[i] #Save the cropped first name in this path
  firstname.save(firstname_path) 

  lastname_path = "/content/gdrive/MyDrive/NTI-Project/Last Names/" +  img_names[i] #Save the cropped first name in this path
  lastname.save(lastname_path)

  nid_path = "/content/gdrive/MyDrive/NTI-Project/National ID/" +  img_names[i] #Save the cropped first name in this path
  nid.save(nid_path) 

In [10]:
def get_paths_images(folder_name):
  img_paths = {} #Images paths 
  img_names = [] #Images names

  folder = "/content/gdrive/MyDrive/NTI-Project/" + folder_name
  for root, dirs, files in os.walk(folder):
    path = root.split(os.sep)
    for index, file in enumerate(files):
        img_names = [ f for f in listdir(root) if isfile(join(root,f)) ]
        img_paths[index] = join(root,img_names[index])
    
    return img_paths, img_names

In [13]:
last_names  = []

img_paths, img_names = get_paths_images('Last Names')
  
for i in range(len(img_paths)):
  #Read the image 
  image = cv2.imread(img_paths[i])
    
  #Preprocessing
  image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  (first_thresh, first_blackAndWhite) = cv2.threshold(image_gray, 100, 255, cv2.THRESH_BINARY)

    
  #Extract and append info
  extracted_info = pytesseract.image_to_string(first_blackAndWhite, lang='ara') 
  last_names.append(extracted_info[:-2])

In [14]:
last_names

['',
 'عياش حكيم قعقاع',
 'عراد حسنين تغلب',
 'هريرة معاذ سلمة',
 'منصور فارس مطر',
 'حسين حمدان ساجع',
 'حزام حمدان مبروك',
 'أمين فتيح عاطف',
 'عقلة ساطع كنان',
 'شبيب ساطع حميد']

In [19]:
#If we applied tesseract on first names it doesn't get good accuracy at all
temp  = []

img_paths, img_names = get_paths_images('First Names')
  
for i in range(len(img_paths)):
  #Read the image 
  image = cv2.imread(img_paths[i])
    
  #Preprocessing
  image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  (first_thresh, first_blackAndWhite) = cv2.threshold(image_gray, 100, 255, cv2.THRESH_BINARY)

    
  #Extract and append info
  extracted_info = pytesseract.image_to_string(first_blackAndWhite, lang='ara') 
  temp.append(extracted_info[:-2])

temp

['', '', '', '', '', '', '', 'عبدالبصير', 'نهيان', '']

In [27]:
!pip install ArabicOcr

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [23]:
#We'll try arabicocr then

first_names = []
img_paths, img_names = get_paths_images("First Names")

for i in range(len(img_paths)):
	out_image = 'out.jpg'
	results = arabicocr.arabic_ocr(img_paths[i],out_image)
	first_names.append(results[0][-2])

[INFO] OCR'ing input image...


[INFO] 0.9963: لبانة
[INFO] OCR'ing input image...


[INFO] 0.4064: رفيف
[INFO] OCR'ing input image...


[INFO] 0.8943: ناشئة
[INFO] OCR'ing input image...


[INFO] 0.7163: بيان
[INFO] OCR'ing input image...


[INFO] 0.9909: جمانة
[INFO] OCR'ing input image...


[INFO] 0.8943: منيرة
[INFO] OCR'ing input image...


[INFO] 0.7851: مجير
[INFO] OCR'ing input image...


[INFO] 0.9998: عبد
[INFO] 0.7167: البصير
[INFO] OCR'ing input image...


[INFO] 0.7263: نهيان
[INFO] OCR'ing input image...
[INFO] 0.9916: فتيح


In [24]:
first_names 

['لبانة',
 'رفيف',
 'ناشئة',
 'بيان',
 'جمانة',
 'منيرة',
 'مجير',
 'عبد',
 'نهيان',
 'فتيح']

In [25]:
#Now we can combine names to be a full name

full_names = []
for i in range(len(first_names)):
  fullname = first_names[i] + " " + last_names[i]
  full_names.append(fullname)

In [26]:
full_names # Almost Perfecto!

['لبانة ',
 'رفيف عياش حكيم قعقاع',
 'ناشئة عراد حسنين تغلب',
 'بيان هريرة معاذ سلمة',
 'جمانة منصور فارس مطر',
 'منيرة حسين حمدان ساجع',
 'مجير حزام حمدان مبروك',
 'عبد أمين فتيح عاطف',
 'نهيان عقلة ساطع كنان',
 'فتيح شبيب ساطع حميد']

In [ ]:
nid = []
img_paths, img_names = get_paths_images("National ID")

for i in range(len(img_paths)):
	img = cv2.imread(img_paths[i], cv2.IMREAD_COLOR)
	
	id_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
	out_image='out.jpg'

	results = arabicocr.arabic_ocr(img_paths[i],out_image)
	nid.append(results)

In [ ]:
#Before preprocessing
from google.colab.patches import cv2_imshow
import cv2
img = cv2.imread('out.jpg', cv2.IMREAD_UNCHANGED)
cv2_imshow(img)